In [1]:
import os

from rag_tool import construct_rag_tool

from tavily_utils import initialize_web_search_tool
from tavily_utils import execute_web_search

from router import router_tool

from llm_utils import initialize_llm

from agents import initialize_router_agent 
from agents import initialize_retriever_agent
from agents import initialize_answer_grader_agent
from agents import initialize_grader_agent
from agents import initialize_hallucination_grader_agent

from tasks import initialize_router_task
from tasks import initialize_answer_task
from tasks import initialize_retriever_task
from tasks import initialize_grader_task
from tasks import initialize_hallucination_task

from crew_operations import orchestrate_work_flow
from crew_operations import execute_work_flow

from config import GROQ_API_KEY
from config import TAVILY_API_KEY

### Set API Environment Variables

In [2]:
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

### RAG Tool

In [3]:
'''
Input Parameters to RAG Tool

- URL: https://aclanthology.org/2020.wmt-1.52.pdf
'''

rag_tool = construct_rag_tool(urls=["https://aclanthology.org/2020.wmt-1.52.pdf"])

/Users/apple/miniconda3/envs/venv_langchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-08-25 23:55:04,494 - 8378616832 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 7 0 (offset 0)
2024-08-25 23:55:04,495 - 8378616832 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 15 0 (offset 0)
2024-08-25 23:55:04,495 - 8378616832 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 22 0 (offset 0)
2024-08-25 23:55:04,496 - 8378616832 - _utils.py-_utils:463 - WARNING: Ignoring wrong pointing object 61 0 (offset 0)
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


In [4]:
'''
Infer from the RAG Tool
'''
rag_tool.run("What is the BLEU Score of the ensemble model of Hindi to Marathi translation?")

Using Tool: Search a PDF's content


'Relevant Content:\nsubmissions.5 ConclusionThis paper describes the techniques involved inour system submitted for the WMT20 Similar Lan-guage Translation task by Infosys. This winningHindi-Marathi translation system is built based onNMT and evaluated based on the metric, BLEU.The domain-based data preprocessing and ﬁl-tering techniques eases model learning. Adoptingnovel approach of anonymizing language agnostictokens aided our system to focus more on tokensthat matters in the translation. It is highly observedthat the ratio of monolingual data used against bi-text data plays a vital role in back-translated mod-els. Improving translation accuracy and languageﬂuency by utilizing all available out-of-domainmonolingual corpora to the maximum effect can befurther explored.ReferencesDzmitry Bahdanau, Kyunghyun Cho, and Yoshua Ben-gio. 2015.Neural machine translation by jointlylearning to align and translate. In3rd InternationalConference on Learning Representations, ICLR, SanDiego, USA.Pa

### Web Search Tool

In [5]:
'''
Initialize Web search tool using Tavily
'''
web_search_tool = initialize_web_search_tool()

In [6]:
'''
Execute web search for the specified content
'''
execute_web_search(web_search_tool=web_search_tool, question="Which is the best World Machine Translation model built in 2020?")

[{'url': 'https://ai.meta.com/blog/introducing-many-to-many-multilingual-machine-translation/',
  'content': "October 19, 2020. Share on Facebook. Share on Twitter. Facebook AI is introducing, M2M-100 the first multilingual machine translation (MMT) model that translates between any pair of 100 languages without relying on English data. It's open sourced here. When translating, say, Chinese to French, previous best multilingual models train on ..."},
 {'url': 'https://ai.meta.com/blog/the-first-ever-multilingual-model-to-win-wmt-beating-out-bilingual-models/',
  'content': 'For the first time, a single multilingual model has outperformed the best bilingual models across 10 out of 14 language pairs to win WMT, a prestigious machine translation competition. Our single multilingual model provided the best translations for both low-and high-resource languages, showing that the multilingual approach is the future of MT.'},
 {'url': 'https://ai.meta.com/blog/nllb-200-high-quality-machine-tra

### Agentic RAG Design Pattern

#### Large Language Model initialization

In [7]:
llm = initialize_llm()

#### Agents Construction

In [8]:
Router_Agent = initialize_router_agent(llm=llm)
Answer_Grader_Agent = initialize_answer_grader_agent(llm=llm)
Grader_Agent = initialize_grader_agent(llm=llm)
Hallucination_Grader_Agent = initialize_hallucination_grader_agent(llm=llm)
Retreiver_Agent = initialize_retriever_agent(llm=llm)

#### Tasks Construction

In [9]:
Router_Task = initialize_router_task(agent=Router_Agent, tool=router_tool)
Retreiver_Task = initialize_retriever_task(agent=Retreiver_Agent, task=Router_Task)
Grader_Task = initialize_grader_task(agent=Grader_Agent, task=Retreiver_Task)
Hallucination_Task = initialize_hallucination_task(agent=Hallucination_Grader_Agent, task=Grader_Task)
Answer_Task = initialize_answer_task(agent=Answer_Grader_Agent, task=Hallucination_Task)


#### CREW AI Operations

In [10]:
inputs = {"question":"Describe the hindi - marathi translation model?"}

rag_crew = orchestrate_work_flow(agents=[Router_Agent, Retreiver_Agent, Grader_Agent, Hallucination_Grader_Agent, Answer_Grader_Agent],
    tasks=[Router_Task, Retreiver_Task, Grader_Task, Hallucination_Task, Answer_Task])
result = execute_work_flow(rag_crew=rag_crew, inputs=inputs)

print("Final Result extracted from the Agentic RAG")
print(result)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: 
Analyze the question Describe the hindi - marathi translation model? for keywords. Determine whether it requires a vector store search or a web search. 
Respond with either 'vectorstore' or 'websearch' as a single word, without additional explanations.



> Entering new CrewAgentExecutor chain...
Thought: The question is asking about a specific translation model, which is a topic in the field of Natural Language Processing (NLP). The question mentions Hindi and Marathi, which are both languages.

Action: router_tool
Action Input: {"question": "Describe the hindi - marathi translation model?"}
 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: 
Based on the router's output, retrieve the necessary information for the question Describe the hindi - marathi translation model?. 
Use the 'web_search_tool' if 